In [1]:
import math
import collections
import heapq
import bisect
import functools
from typing import *

# 6315. Count the Number of Vowel Strings in Range

In [2]:
class Solution:
    def vowelStrings(self, words: List[str], left: int, right: int) -> int:
        vowels = "aeiou"
        words = words[left:right+1]
        ret = 0
        for word in words:
            if word[0] in vowels and word[-1] in vowels:
                ret += 1
        return ret

# 6316. Rearrange Array to Maximize Prefix Score

In [3]:
class Solution:
    def maxScore(self, nums: List[int]) -> int:
        nums.sort(reverse=True)
        sums = 0
        ret = 0
        for num in nums:
            sums += num
            if sums > 0:
                ret += 1
            else:
                break
        return ret

# 6317. Count the Number of Beautiful Subarrays

In [37]:
class Solution:
    def beautifulSubarrays(self, nums: List[int]) -> int:
        vis = {0:1}
        sums = 0
        ret = 0
        for i, num in enumerate(nums):
            sums ^= num
            ret += vis.get(sums, 0)
            vis[sums] += 1
        print(vis)
        return ret
        

In [38]:
nums = [0, 0]

In [39]:
a = Solution().beautifulSubarrays(nums)
a

{0: 3}


3

# 6318. Minimum Time to Complete All Tasks

In [213]:
class Solution:
    def findMinimumTime(self, tasks: List[List[int]]) -> int:
        sums = [0] * 8
        for s,e,d in tasks:
            sums[s] += 1
            sums[e+1] -= 1
        for i in range(1, len(sums)):
            sums[i] += sums[i-1]
        h = []
        for i, n in enumerate(sums):
            heapq.heappush(h, [-n, i])
        
        ret = 0
        while h:
            s, i = heapq.heappop(h)
            if -s > sums[i]:
                heapq.heappush(h, [-sums[i], i])
                continue
                
            flag = False
            a = []
            for j, (s, e, d) in enumerate(tasks):
                if s <= i <= e and d:
                    if d == 1:
                        a.append(j)                    
                    tasks[j][2] -= 1
                    flag = True
            
            for f in a:
                for j in range(tasks[f][0], tasks[f][1]+1):
                    sums[j] -= 1
            
            ret += flag
        return ret

In [230]:
tasks = tasks = [[1,3,2], [1,3,1]]

In [231]:
a = Solution().findMinimumTime(tasks)
a

2

In [233]:
# 贪心
class Solution:
    def findMinimumTime(self, tasks: List[List[int]]) -> int:
        tasks.sort(key=lambda x:x[1])
        vis = [False] * 2002
        for start, end, d in tasks:
            t = d - sum(vis[start:end+1])
            if t > 0:
                for j in range(end, start-1, -1):
                    if not vis[j]:
                        vis[j] = True
                        t -= 1
                        if t == 0:
                            break
        
        return sum(vis)

In [12]:
# 差分约束
class Solution:
    def findMinimumTime(self, tasks: List[List[int]]) -> int:
        # 令Xi为i时间段下电脑需要运行的次数
        t = max(task[1] for task in tasks)
        edges = [[] for _ in range(t+2)]
        values = [[] for _ in range(t+2)]
        
        # Xi-1 <= Xi
        for i in range(t):
            edges[i].append(i+1)
            values[i].append(0)
        
        # Xi <= Xi-1 + 1
        for i in range(1, t+1):
            edges[i].append(i-1)
            values[i].append(1)
        
        # Xe - Xs-1 >= d -> Xs-1 <= Xe - d
        for s, e, d in tasks:
            edges[s-1].append(e)
            values[s-1].append(-d)
        
        # 起始源点
        for i in range(t+1):
            edges[t+1].append(i)
            values[t+1].append(0)
        
        # SPFA算法
        dis = [float("inf")] * (t+2)
        dis[-1] = 0
        q = collections.deque([[t+1, 0]])
        while q:
            i, s = q.popleft()
            for j, nxt in enumerate(edges[i]):
                if values[i][j] + s < dis[nxt]:
                    q.append([nxt, s+values[i][j]])
                    dis[nxt] = values[i][j] + s
                    
        return dis[0] - dis[t]

In [37]:
tasks = [[2,3,1],[4,5,1],[1,5,2]]

In [46]:
a = Solution().findMinimumTime(tasks)
a

2

In [81]:
class Solution:
    def findMinimumTime(self, tasks: List[List[int]]) -> int:
        tasks.sort(key=lambda x:x[1])
        q = [(-1, -1, 0)]
        print(tasks)
        for s, e, d in tasks:
            idx = bisect.bisect_left(q, (s, )) - 1
            d -= q[-1][2] - q[idx][2]
            if s <= q[idx][1]:
                d -= q[idx][1] - s + 1
            print(s, e, d, q, idx)
            if d <= 0:
                continue
            while q and e <= d + q[-1][1]:
                l, r, _ = q.pop()
                d += r - l + 1
            q.append((e-d + 1, e, d+q[-1][2]))
        return q[-1][2]

In [82]:
tasks = [[14,20,5],[2,18,7],[6,14,1],[3,16,3]]

In [83]:
a = Solution().findMinimumTime(tasks)
a

[[6, 14, 1], [3, 16, 3], [2, 18, 7], [14, 20, 5]]
6 14 1 [(-1, -1, 0)] 0
3 16 2 [(-1, -1, 0), (14, 14, 1)] 0
2 18 4 [(-1, -1, 0), (14, 16, 3)] 0
14 20 0 [(-1, -1, 0), (12, 18, 7)] 1


7